In [ ]:
# Install boto3 if needed
# !pip install boto3

import boto3
import time

# Step 1: Start session with credentials
# Add AWS credentials here
session = boto3.Session(
    aws_access_key_id='',
    aws_secret_access_key='',
    region_name=''
)

# Step 2: Connect to DynamoDB
dynamodb = session.resource('dynamodb')

# Step 3: Create the table
table = dynamodb.create_table(
    TableName='StockMktPro',
    AttributeDefinitions=[
        {
            'AttributeName': 'Company Name',
            'AttributeType': 'S'  # String
        },
        {
            'AttributeName': 'Date',
            'AttributeType': 'S'  # String
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'Company Name',
            'KeyType': 'HASH'  # Partition Key
        },
        {
            'AttributeName': 'Date',
            'KeyType': 'RANGE'  # Sort Key
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print("⏳ Table creation started. Waiting for it to become ACTIVE...")

# Step 4: Wait until table becomes active
table.meta.client.get_waiter('table_exists').wait(TableName='StockMktPro')

# Step 5: Confirm status
table = dynamodb.Table('StockMktPro')
print("✅ Table is now:", table.table_status)


⏳ Table creation started. Waiting for it to become ACTIVE...
✅ Table is now: ACTIVE


In [ ]:
!pip install kafka-python boto3

from kafka import KafkaConsumer
from json import loads
import boto3
from decimal import Decimal



In [ ]:
consumer = KafkaConsumer(
    'demo',
    bootstrap_servers=[''],  # Use your EC2 public IP
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)

def convert_floats(obj):
    for key, value in obj.items():
        if isinstance(value, float):
            obj[key] = Decimal(str(value))
    return obj


for msg in consumer:
    data = msg.value

    # Ensure required keys are present
    if 'Company Name' in data and 'Date' in data:
        data['Date'] = str(data['Date'])  # Ensure Date is string
        data = convert_floats(data)       # Convert float fields
        table.put_item(Item=data)         # Insert into DynamoDB
        print("✅ Inserted:", data)
    else:
        print("⚠️ Skipping record (missing Company Name or Date):", data)


✅ Inserted: {'Date': '19-04-2023', 'Open': Decimal('212.4686363'), 'High': Decimal('216.3105301'), 'Low': Decimal('211.9311609'), 'Close': Decimal('214.6881714'), 'Volume': 15898100, 'Company Name': 'Meta'}
✅ Inserted: {'Date': '20-04-2023', 'Open': Decimal('212.478576'), 'High': Decimal('215.733241'), 'Low': Decimal('211.7719151'), 'Close': Decimal('212.0705109'), 'Volume': 16475400, 'Company Name': 'Meta'}
✅ Inserted: {'Date': '21-04-2023', 'Open': Decimal('209.2239363'), 'High': Decimal('212.4089224'), 'Low': Decimal('208.5968867'), 'Close': Decimal('211.8913574'), 'Volume': 17717200, 'Company Name': 'Meta'}
✅ Inserted: {'Date': '24-04-2023', 'Open': Decimal('212.6776335'), 'High': Decimal('212.9165132'), 'Low': Decimal('209.7215796'), 'Close': Decimal('211.7918091'), 'Volume': 15750900, 'Company Name': 'Meta'}
✅ Inserted: {'Date': '25-04-2023', 'Open': Decimal('209.8310657'), 'High': Decimal('210.2689889'), 'Low': Decimal('206.1583728'), 'Close': Decimal('206.5764008'), 'Volume': 1

ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-2.1.5, node_id=0 host=44.211.126.1:9092 <connected> [IPv4 ('44.211.126.1', 9092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-2.1.5, node_id=0 host=44.211.126.1:9092 <connected> [IPv4 ('44.211.126.1', 9092)]>: Closing connection. KafkaConnectionError: socket disconnected
ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: KafkaConnectionError: socket disconnected
ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: NodeNotReadyError: 0
ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-2.1.5, node_id=0 host=44.211.126.1:9092 <connecting> [IPv4 ('44.211.126.1', 9092)]>: Connect attempt returned error 111. Disconnecting.
ERROR:kafka.conn:<BrokerConnection client_id=kafka-python-2.1.5, node_id=0 host=44.211.126.1:9092 <connecting> [IPv4 ('44.211.126.1', 9092)]>: Closing connection. KafkaConnectionError: 111 ECONNREFUSED
ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: NodeNotReadyErro

KeyboardInterrupt: 